# MLP con Dropout y Early Stopping para el Dataset Heart Disease UCI

### Paso 1: Carga de Datos

In [ ]:

import pandas as pd
import numpy as np

# Cargar los datos
file_path = "heart.csv"  # Cambia esta ruta si es necesario
df = pd.read_csv(file_path)

# Inspeccionar los datos
print("Primeras filas del dataset:")
print(df.head())

# Información del dataset
print("\nInformación del dataset:")
print(df.info())

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


### Paso 2: Exploración y Preprocesamiento de Datos

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dividir en características y variable objetivo
X = df.drop(columns=['target'])
y = df['target']

# Normalizar las características
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Verificar el balance de la variable objetivo
print("\nDistribución de la variable objetivo:")
print(y.value_counts())

# Dividir en conjuntos de entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nTamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)


### Paso 3: Implementación de la MLP con Dropout y Early Stopping

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Crear el modelo MLP con Dropout
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Configurar Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


### Paso 4: Entrenamiento del Modelo

In [ ]:

# Entrenar el modelo
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    batch_size=32, 
    validation_split=0.2, 
    callbacks=[early_stopping], 
    verbose=1
)

# Visualizar las curvas de aprendizaje
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Pérdida
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# Precisión
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()

plt.tight_layout()
plt.show()


### Paso 5: Evaluación del Modelo

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

# Predicciones
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:")
print(conf_matrix)

# Informe de clasificación
print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred))
